# 1. 영어 Word2Vec 만들기
이번에는 영어 데이터를 다운로드 받아 직접 Word2Vec 작업을 진행해보도록 하겠습니다. 파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

### 1) 훈련 데이터 이해하기

In [29]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f5695104890>)

In [30]:
!ls -l

total 97872
-rw-r--r-- 1 root root 25678377 Feb 24 01:01 eng_w2v
drwxr-xr-x 1 root root     4096 Feb 22 14:38 sample_data
-rw-r--r-- 1 root root 74533638 Feb 24 01:10 ted_en-20160408.xml


### 훈련데이터 전처리
훈련 데이터 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아니라, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거해야 합니다.

In [31]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [32]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)


In [33]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [34]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

In [35]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [36]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [37]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### 3) Word2Vec 훈련시키기

In [38]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [39]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8468674421310425), ('guy', 0.8228563666343689), ('boy', 0.7958941459655762), ('lady', 0.7753374576568604), ('girl', 0.7640807628631592), ('gentleman', 0.7314152717590332), ('kid', 0.7061263918876648), ('soldier', 0.7008715271949768), ('physicist', 0.6867040395736694), ('friend', 0.6829357743263245)]


In [41]:
[('woman', 0.842622697353363), ('guy', 0.8178728818893433), ('boy', 0.7774451375007629), ('lady', 0.7767927646636963), ('girl', 0.7583760023117065), ('gentleman', 0.7437191009521484), ('soldier', 0.7413754463195801), ('poet', 0.7060446739196777), ('kid', 0.6925194263458252), ('friend', 0.6572611331939697)]

[('woman', 0.842622697353363),
 ('guy', 0.8178728818893433),
 ('boy', 0.7774451375007629),
 ('lady', 0.7767927646636963),
 ('girl', 0.7583760023117065),
 ('gentleman', 0.7437191009521484),
 ('soldier', 0.7413754463195801),
 ('poet', 0.7060446739196777),
 ('kid', 0.6925194263458252),
 ('friend', 0.6572611331939697)]

### 4) Word2Vec 모델 저장하고 로드하기

In [42]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [43]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8468674421310425), ('guy', 0.8228563666343689), ('boy', 0.7958941459655762), ('lady', 0.7753374576568604), ('girl', 0.7640807628631592), ('gentleman', 0.7314152717590332), ('kid', 0.7061263918876648), ('soldier', 0.7008715271949768), ('physicist', 0.6867040395736694), ('friend', 0.6829357743263245)]


In [44]:
[('woman', 0.842622697353363), ('guy', 0.8178728818893433), ('boy', 0.7774451375007629), ('lady', 0.7767927646636963), ('girl', 0.7583760023117065), ('gentleman', 0.7437191009521484), ('soldier', 0.7413754463195801), ('poet', 0.7060446739196777), ('kid', 0.6925194263458252), ('friend', 0.6572611331939697)]

[('woman', 0.842622697353363),
 ('guy', 0.8178728818893433),
 ('boy', 0.7774451375007629),
 ('lady', 0.7767927646636963),
 ('girl', 0.7583760023117065),
 ('gentleman', 0.7437191009521484),
 ('soldier', 0.7413754463195801),
 ('poet', 0.7060446739196777),
 ('kid', 0.6925194263458252),
 ('friend', 0.6572611331939697)]